In [1]:
%%html
<style> .container { width: 100% !important; }</style>

In [3]:
!pip install --upgrade pip

Defaulting to user installation because normal site-packages is not writeable


In [1]:
import pandas as pd
pd.set_option('max_colwidth', 200)  # Set a very large value
pd.set_option('max_row', 200)  # Set a very large value

def add_borders(val):
    return 'border: 1px solid black;'

style_df = lambda df : df.style.applymap(add_borders)

In [9]:
#locations_df = pd.read_json("data2.json")
locations_df = pd.read_json("./LocationsParsed.json")

----- Data Example

In [10]:
locations_df.head(1).T

,0
database,ushvr12
db_instance,usinnovationredshift
db_node_name,us-innovation-redshift.c8ziwm1qxh67.us-east-1.redshift.amazonaws.com
dbdesc,dev
host,odp-us-innovation-postgres-db.cxzrm3enxxmy.us-east-1.rds.amazonaws.com
loc_class,redshift
loc_db_user,504005930
loc_description,tredp
loc_name,tredp
loc_remote_login,isc_dev


In [11]:
pd.DataFrame (locations_df.columns, columns=["columns"])

,columns
0,database
1,db_instance
2,db_node_name
3,dbdesc
4,host
5,loc_class
6,loc_db_user
7,loc_description
8,loc_name
9,loc_remote_login


In [12]:
set (locations_df.dbdesc)

{'Finance Nprod', 'US hvr prod', 'Us innovation', 'dev', 'fin-prod'}

---------------

-- Analysis --

--------------

In [13]:
locations_prod_df = locations_df[locations_df['dbdesc'].isin(['US hvr prod','fin-prod'])]

-- how many locations are using our agents

locations_prod_df[['loc_remote_node','loc_remote_port']].drop_duplicates().sort_values(by=['loc_remote_node','loc_remote_port'])

In [14]:
print ('Number of locations per hvr meta database')

style_df(
locations_prod_df.groupby(['dbdesc', 'database'])['loc_name'].count().reset_index().set_index(['dbdesc','database'])
)


Number of locations per hvr meta database


In [15]:
print ('number of agents per node (represented as port) - count of locations')

style_df (
locations_prod_df.pivot_table(index='loc_remote_node', columns=['loc_remote_port'], values='loc_name', aggfunc='count').fillna('').astype(str).apply(lambda x: x.str.rstrip('.0'))
)



number of agents per node (represented as port) - count of locations


loc_remote_port,0,4343,4344,6080,8080,8088,8443,9090,9995,18081
loc_remote_node,,,,,,,,,,
,22,,,,,,,,,
10.229.1.113,,1,,,,,,,,
10.229.2.236,,,,,4,,,,,
10.242.109.196,,448,,8,5,,1,,1,
10.242.109.235,,,,,3,,,,,
10.242.109.242,,,,,,,4,26,4,1
10.242.109196,,3,,,,,,,,
10.242.112.130,,34,,,,,,,,
10.242.112.153,,6,,1,,86,1,186,,


In [ ]:
-- rac://cgehedq2-scan.am.health.ge.com:1525 repeated multiple times is it same agent ?

In [16]:
style_df (
            locations_prod_df.pivot_table(index='loc_remote_node', columns=['loc_class'], values='loc_name', aggfunc='count').fillna('').astype(str).apply(lambda x: x.str.rstrip('.0'))
         )

loc_class,file,file_locdir,file_s3s,file_sftp,greenplum,kafka,mysql,oracle,postgresql,redshift,salesforce,sqlserver,teradata
loc_remote_node,,,,,,,,,,,,,
,,7,84,4,,3,2,88,24,,3,,5
10.229.1.113,,,1,,,,,,,,,,
10.229.2.236,,,2,,,,,,,2,,,
10.242.109.196,4,315,7,15,2,4,,29,51,28,,,17
10.242.109.235,,,,,,,,,,3,,,
10.242.109.242,,,12,,,,,,19,4,,,
10.242.109196,,3,,,,,,,,,,,
10.242.112.130,,,,,,,,,1,,3,3,
10.242.112.153,,153,6,,,2,,2,48,43,,,17


In [17]:
style_df (
locations_prod_df.pivot_table(index='loc_remote_node', columns=['dbdesc','database'], values='loc_name', aggfunc='count').fillna('').astype(str).apply(lambda x: x.str.rstrip('.0'))
)